In [1]:
!pip3 install numpy matplotlib scipy sklearn hmmlearn simplejson eyed3 pydub
# !pip3 install pyAudioAnalysis
# !pip3 install plot_metrics
!pip3 install keras
!pip3 install boto

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.5/135.5 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.1/246.1 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 KB 23.3 MB/s eta 0:00:00
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=66c875105115efc83a148058bad3cf1d01566c63750a57160bdb5e91cafff0ec
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4

Import all the required libraries

In [2]:
import fnmatch
import os
import zipfile
import re

import numpy as np
from numpy.lib import stride_tricks
import os
import tarfile

import pandas as pd

import random
import boto

from sklearn.metrics import confusion_matrix, roc_curve, auc
# from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd /content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data

/content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data


In [5]:
# !wget -r -np -nH --cut-dirs=3 -R index.html --user=daicwozuser --no-check-certificate https://dcapswoz.ict.usc.edu/wwwedaic/data/

In [6]:
dir_name = '/content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data'

# directory where audio and transcripts folders will be created
out_dir = '/content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data/raw'


In [7]:
# !wget -r -np -nH --cut-dirs=3 -R index.html --user=daicwozuser --no-check-certificate https://dcapswoz.ict.usc.edu/wwwedaic/labels/

In [8]:
# /content/drive/MyDrive/Colab Notebooks/Depression Analysis/train_split.csv
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Depression Analysis/train_split.csv')

# /content/drive/MyDrive/Colab Notebooks/Depression Analysis/test_split.csv
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Depression Analysis/test_split.csv')

# /content/drive/MyDrive/Depression_Analysis_Research/dev_split.csv
dev_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Depression Analysis/dev_split.csv')

In [9]:
print(train_df.shape)
print(test_df.shape)
print(dev_df.shape)

(163, 6)
(56, 6)
(56, 6)


In [10]:
print('train_df.head()', train_df.head())
print('test_df.head()', test_df.head())
print('dev_df.head()', dev_df.head())

train_df.head()    Participant_ID   Gender  PHQ_Binary  PHQ_Score  PCL-C (PTSD)  PTSD Severity
0             302     male           0          4             0             28
1             303   female           0          0             0             17
2             304   female           0          6             0             20
3             305     male           0          7             0             28
4             307  female            0          4             0             23
test_df.head()    Participant_ID  Gender  PHQ_Binary  PHQ_Score  PCL-C (PTSD)  PTSD Severity
0             600  female           0          5             0           23.0
1             602  female           1         13             1           67.0
2             604    male           1         12             0           30.0
3             605    male           0          2             0           23.0
4             606  female           0          5             0           46.0
dev_df.head()    Participan

In [11]:
def in_split(partic_id, df):
    """
    Returns True if the participant is in the AVEC training split
    (aka participant's we have depression labels for)
    """
    return partic_id in set(df['Participant_ID'].values)

def get_depression_label(partic_id, df):
    """
    Returns participant's PHQ8 Binary label. 1 representing depression;
    0 representing no depression.
    """
    return df.loc[df['Participant_ID'] == partic_id]['PHQ_Score'].item()


In [14]:
def build_class_dictionaries(dir_name, df):
    
    print("dir name: " + dir_name)

    depressed_dict = dict()
    normal_dict = dict()
    
    for subdir, dirs, files in os.walk(dir_name):
        for file in files:
            if fnmatch.fnmatch(file, '*BoVW_openFace_2.1.0_Pose_Gaze_AUs.csv'):
                # print(file)
                regex = re.compile(r'\d+')
                partic_id = [int(x) for x in regex.findall(file)][0]

                if in_split(partic_id, df):
                    file_name = dir_name + '/' + str(partic_id) + '_P/features/' + file
                    
                    df_1 = pd.read_csv(file_name, header=None, delimiter=',')
                    df_1.drop([0], axis=1, inplace=True)
                    mat = np.mat(df_1)

                    depressed = get_depression_label(partic_id, df)  # 1 if True
                    if depressed:
                        # print("Depressed")
                        # print(partic_id)
                        # print(mat)
                        depressed_dict[partic_id] = mat
                    elif not depressed:
                        normal_dict[partic_id] = mat
    return depressed_dict, normal_dict

In [26]:
dir_name = '/content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data/raw/vid_features'
depressed_train_dict, normal_train_dict = build_class_dictionaries(dir_name, train_df)

dir name: /content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data/raw/vid_features


In [27]:
dir_name = '/content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data/raw/vid_features'
depressed_test_dict, normal_test_dict = build_class_dictionaries(dir_name, test_df)

dir name: /content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data/raw/vid_features


In [28]:
dir_name = '/content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data/raw/vid_features'
depressed_dev_dict, normal_dev_dict = build_class_dictionaries(dir_name, dev_df)

dir name: /content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data/raw/vid_features


In [29]:
del(dir_name)

In [30]:
print(len(depressed_train_dict))
print(len(normal_train_dict))

print(len(depressed_test_dict))
print(len(normal_test_dict))

print(len(depressed_dev_dict))
print(len(normal_dev_dict))

139
24
47
9
47
9


In [31]:
def create_y_set(depressed_dict, normal_dict, df):
    """
    Building an array that describes if a participant is depressed or not
    """
    merged_dict = {**depressed_dict, **normal_dict}

    y_func = list()
    part_id = list()

    for id in df['Participant_ID']:
        part_id.append(id)

    # print(len(part_id))
    for key, value in merged_dict.items():
        if key in part_id:
            # print(key)
            x = df.loc[df['Participant_ID']==key, 'PHQ_Score'].iloc[0]
            y_func.append(x)    
    # print(df['Participant_ID'])
    # y_func = pd.DataFrame()
    return y_func, merged_dict



In [32]:
y_train, merged_train_dict = create_y_set(depressed_train_dict, normal_train_dict, train_df)
print(y_train)

[4, 6, 7, 4, 22, 15, 4, 21, 2, 7, 1, 2, 6, 3, 13, 5, 1, 5, 10, 2, 4, 4, 1, 12, 18, 5, 12, 10, 15, 11, 1, 7, 15, 23, 20, 5, 14, 10, 11, 18, 10, 10, 7, 7, 13, 4, 20, 19, 7, 13, 5, 12, 16, 1, 2, 10, 15, 8, 11, 2, 9, 9, 1, 7, 5, 5, 7, 7, 17, 2, 3, 10, 12, 10, 16, 3, 7, 10, 3, 3, 10, 12, 3, 20, 5, 1, 3, 1, 2, 2, 1, 19, 1, 7, 1, 1, 2, 9, 1, 17, 6, 3, 5, 16, 17, 9, 9, 3, 3, 3, 4, 3, 2, 1, 7, 1, 15, 2, 3, 2, 8, 4, 2, 1, 10, 17, 6, 2, 2, 13, 12, 22, 18, 7, 5, 8, 1, 7, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [33]:
y_test, merged_test_dict = create_y_set(depressed_test_dict, normal_test_dict, test_df)
print(y_test)
# print(merged_test_dict)

[5, 13, 12, 2, 5, 7, 3, 4, 6, 9, 22, 2, 2, 9, 15, 16, 19, 17, 19, 3, 5, 7, 12, 8, 11, 16, 19, 9, 2, 5, 7, 8, 12, 6, 19, 13, 11, 3, 13, 16, 20, 10, 7, 7, 15, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [34]:
y_dev, merged_dev_dict = create_y_set(depressed_dev_dict, normal_dev_dict, dev_df)
print(y_dev)

[2, 3, 8, 11, 20, 8, 5, 7, 9, 11, 16, 11, 12, 9, 9, 2, 16, 2, 9, 11, 12, 3, 6, 2, 10, 8, 18, 6, 18, 4, 1, 1, 2, 4, 6, 7, 1, 9, 4, 12, 3, 1, 4, 3, 7, 19, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [24]:
del(depressed_train_dict)
del(depressed_test_dict)
del(depressed_dev_dict)
del(normal_train_dict)
del(normal_test_dict)
del(normal_dev_dict)

In [ ]:
# x_train = np.array(list(merged_train_dict.values()))
# x_test = np.array(list(merged_test_dict.values()))
# x_dev = np.array(list(merged_dev_dict.values()))

In [ ]:
# del(merged_train_dict)
# del(merged_test_dict)
# del(merged_dev_dict)

In [35]:
def data_handling(x, y):

    for i in range(len(x)):
        x[i] = np.ravel(x[i])

    max = -1
    for i in range(len(y)):
        if len(y[i]) > max:
            max = len(y[i])
    
    for i in range(len(x)):
        length = max - len(x[i])
        x[i] = np.pad(x[i], (0, length), 'constant')
        del(length)

    arr = np.array([])
    for element in x:
        arr = np.concatenate((arr, element))
        del(element)
    
    del(x)

    return arr, max

In [36]:
# print(x_test[0].shape)

In [37]:
# x_train_1 = x_train[0:81] 
# x_train_2 = x_train[81:163]

# arr_1, max_train = data_handling(x_train_1, x_train)
# del(x_train_1)

# arr_2, max_train = data_handling(x_train_2, x_train)
# del(x_train_2)

# del(x_train)

# arr_1 = np.concatenate((arr_1, arr_2))

# del(arr_2)

# arr_3, max_test = data_handling(x_test, x_test)
# print(max_test)
# del(x_test)

# arr_4, max_dev = data_handling(x_dev, x_dev)
# del(x_dev)

In [38]:
# x_train = arr_1.reshape(163, int(max_train/100), 100)
# del(arr_1)

# x_test = arr_3.reshape(56, int(max_test/100), 100)
# del(arr_3)

# x_dev = arr_4.reshape(56, int(max_dev/100), 100)
# del(arr_4)

In [39]:
# del(max_train)
# del(max_test)
# del(max_dev)

In [40]:
# print(x_train.shape)
# print(x_test.shape)
# print(x_dev.shape)

In [41]:
y_train = np.array(y_train)
y_test = np.array(y_test)
y_dev = np.array(y_dev)

In [42]:
# # create audio directory
# out_dir = '/content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data'
# vid_features_dir = os.path.join(out_dir, 'vid_features_BoVW_openFace_210_Pose_Gaze_AUs')
# if not os.path.exists(vid_features_dir):
#     os.makedirs(vid_features_dir)

In [43]:
%ls
%cd /content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data/vid_features_BoVW_openFace_210_Pose_Gaze_AUs
%ls

aud_features_BoAW_openSMILE_230_MFCC/
raw/
vid_features_BoVW_openFace_210_Pose_Gaze_AUs/
/content/drive/MyDrive/Colab Notebooks/Depression Analysis/Data/vid_features_BoVW_openFace_210_Pose_Gaze_AUs
x_dev_BoVW_openFace_210_Pose_Gaze_AUs.gz
x_test_BoVW_openFace_210_Pose_Gaze_AUs.gz
x_train_BoVW_openFace_210_Pose_Gaze_AUs.gz
y_dev_BoVW_openFace_210_Pose_Gaze_AUs.gz
y_test_BoVW_openFace_210_Pose_Gaze_AUs.gz
y_train_BoVW_openFace_210_Pose_Gaze_AUs.gz


In [44]:
# filename = 'x_train_BoVW_openFace_210_Pose_Gaze_AUs.gz'
# np.savetxt('x_train_BoVW_openFace_210_Pose_Gaze_AUs.gz', x_train.reshape(x_train.shape[0], -1), delimiter=',')
# del(x_train)

# filename = 'x_test_BoVW_openFace_210_Pose_Gaze_AUs.gz'
# np.savetxt('x_test_BoVW_openFace_210_Pose_Gaze_AUs.gz', x_test.reshape(x_test.shape[0], -1), delimiter=',')
# del(x_test)

# filename = 'x_dev_BoVW_openFace_210_Pose_Gaze_AUs.gz'
# np.savetxt('x_dev_BoVW_openFace_210_Pose_Gaze_AUs.gz', x_dev.reshape(x_dev.shape[0], -1), delimiter=',')
# del(x_dev)

In [47]:
filename = 'y_train_BoVW_openFace_210_Pose_Gaze_AUs.gz'
np.savetxt('y_train_BoVW_openFace_210_Pose_Gaze_AUs.gz', y_train.reshape(y_train.shape[0], -1), delimiter=',')

filename = 'y_test_BoVW_openFace_210_Pose_Gaze_AUs.gz'
np.savetxt('y_test_BoVW_openFace_210_Pose_Gaze_AUs.gz', y_test.reshape(y_test.shape[0], -1), delimiter=',')

filename = 'y_dev_BoVW_openFace_210_Pose_Gaze_AUs.gz'
np.savetxt('y_dev_BoVW_openFace_210_Pose_Gaze_AUs.gz', y_dev.reshape(y_dev.shape[0], -1), delimiter=',')